# Import packages

In [1]:
import os
import pandas as pd
from src import features
from src import database
import yfinance as yf

from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
ROOT_DIR = os.getenv('ROOT_DIR')

# Load dataset

In [2]:
# Build the full path to the CSV file
csv_file_path = os.path.join(ROOT_DIR, 'app', 'backend', 'transaction.csv')

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [3]:
df.head()

,id,isinId,brokerId,accountId,date,orderId,quantity,unitPrice
0,1,1,1,1,2023-10-16,1,2.0,652.06
1,2,3,1,2,2023-10-16,1,15.0,130.30
2,3,3,1,2,2023-10-24,1,1.0,126.95
3,4,1,1,1,2023-10-24,1,1.0,626.81
4,5,3,1,2,2023-11-13,1,1.0,131.87


# Add features

In [4]:
df_features = features.add_column_total(df)
df_features.head()

,id,isinId,brokerId,accountId,date,orderId,quantity,unitPrice,total
0,1,1,1,1,2023-10-16,1,2.0,652.06,1304.12
1,2,3,1,2,2023-10-16,1,15.0,130.30,1954.50
2,3,3,1,2,2023-10-24,1,1.0,126.95,126.95
3,4,1,1,1,2023-10-24,1,1.0,626.81,626.81
4,5,3,1,2,2023-11-13,1,1.0,131.87,131.87


In [5]:
ticker_symbol = 'MKL'  # Replace with the actual ticker symbol corresponding to the ISIN

# Fetching the stock data using yfinance
stock = yf.Ticker(ticker_symbol)

# Get the current price
current_price = stock.info['previousClose']
current_price

1550.27

# Database

In [7]:
# Create database from csv
database.create_from_csv(
    os.path.join(ROOT_DIR, 'app', 'backend', 'transaction.csv'),
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)

In [11]:
# List all tables of the database
all_tables = database.list_tables(
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)
print(all_tables)

['account', 'broker', 'order', 'security', 'transaction']
